In [1]:
from __future__ import unicode_literals, print_function, division
from io import open
import re
import pickle
import unicodedata
import string
import re
import random
import pandas as pd
import numpy as np
from gensim.models.keyedvectors import KeyedVectors
import operator
from sklearn.model_selection import train_test_split
from nltk.translate.bleu_score import sentence_bleu

import torch
import torch.nn as nn
from torch import optim
import torch.nn.functional as F
from torch.utils.data import DataLoader

In [2]:
if torch.cuda.is_available():
    device = torch.device('cuda:0')
else:
    device = torch.device('cpu')

In [3]:
skip_training = False

In [4]:
import io

def load_vectors(fname):
    fin = io.open(fname, 'r', encoding='utf-8', newline='\n', errors='ignore')
    n, d = map(str, fin.readline().split())
    data = {}
    for line in fin:
        tokens = line.rstrip().split(' ')
        data[tokens[0]] = map(float, tokens[1:])
    return data

In [5]:
# load mk embeddings
# mk_embeddings = KeyedVectors.load('data/embeddings/mk_embeddings.bin')
# mk_weights = torch.FloatTensor(mk_embeddings.wv.vectors).to(device)

In [6]:
# load fi embeddings
# fi_embeddings = KeyedVectors.load('data/embeddings/fi_embeddings.bin')
# i_weights = torch.FloatTensor(fi_embeddings.wv.vectors).to(device)

In [7]:
# fi_embedding = KeyedVectors.load_word2vec_format('data/embeddings/cc.fi.300.vec', limit=500000)
# mk_embedding = KeyedVectors.load_word2vec_format('data/embeddings/cc.mk.300.vec', limit=500000)

In [8]:
# mk_embeddings = load_vectors('data/embeddings/cc.mk.300.vec')
# fi_embeddings = load_vectors('data/embeddings/cc.fi.300.bin')

In [9]:
SOS_token = 0
EOS_token = 1
MAX_LENGTH = 10

# class Lang:
#     def __init__(self, name):
#         self.name = name
#         self.word2index = {}
#         self.word2count = {}
#         self.index2word = {0: "SOS", 1: "EOS"}
#         self.n_words = 2  # Count SOS and EOS

#     def addSentence(self, sentence):
# #         for word in sentence.split(' '):
#         for word in re.findall(r"[\w'-]+|[.,!?;]", sentence):
#             if word[0] == '-' and len(word) > 1:
#                 self.addWord(word[1:])
#             elif word[-1] == '-' and len(word) > 1:
#                 self.addWord(word[:-1])
#             else:
#                 self.addWord(word)

#     def addWord(self, word):
#         if word not in self.word2index:
#             if self.name == 'fin_val':
#                 if word in fi_embeddings.wv.vocab:
#                     embedding_index = fi_embeddings.wv.vocab[word].index
#                 else:
#                     fi_embeddings.wv.add(word, np.random.normal(scale=0.6, size=(300, )), replace=False)
#                     embedding_index = fi_embeddings.wv.vocab[word].index
                    
#             elif self.name == 'mk_val':
#                 if word in mk_embeddings.wv.vocab:
#                     embedding_index = mk_embeddings.wv.vocab[word].index
#                 else:
#                     mk_embeddings.wv.add(word, np.random.normal(scale=0.6, size=(300, )), replace=False)
#                     embedding_index = mk_embeddings.wv.vocab[word].index
                    
                    
#             self.word2index[word] = embedding_index
#             self.word2count[word] = 1
#             self.index2word[embedding_index] = word
#             self.n_words += 1
#         else:
#             self.word2count[word] += 1


class Lang:
    def __init__(self, name):
        self.name = name
        self.word2index = {}
        self.word2count = {}
        self.index2word = {0: "SOS", 1: "EOS"}
        self.n_words = 2  # Count SOS and EOS

    def addSentence(self, sentence):
        for word in sentence.split(' '):
            self.addWord(word)

    def addWord(self, word):
        if word not in self.word2index:
            self.word2index[word] = self.n_words
            self.word2count[word] = 1
            self.index2word[self.n_words] = word
            self.n_words += 1
        else:
            self.word2count[word] += 1

In [10]:
def unicodeToAscii(s):
    return ''.join(
        c for c in unicodedata.normalize('NFD', s)
        if unicodedata.category(c) != 'Mn'
    )

# Lowercase, trim, and remove non-letter characters
def normalizeString(s):
#     s = unicodeToAscii(s.lower().strip())
    s = re.sub(r"([.!?])", r" \1", s)
    s = re.sub(r"[^a-zA-Z.!?]+", r" ", s)
    return s

In [11]:
def readLangs(lang1, lang2, reverse=False):
    print("Reading lines...")

    # Read the file and split into lines
    lines = open('data/%s-%s.txt' % (lang1, lang2), encoding='utf-8').\
        read().strip().split('\n')

    
    # Split every line into pairs and normalize
    pairs = [[s for s in l.split('\t')] for l in lines]
    
    # Reverse pairs, make Lang instances
    if reverse:
        pairs = [list(reversed(p)) for p in pairs]
        input_lang = Lang(lang2)
        output_lang = Lang(lang1)
    else:
        input_lang = Lang(lang1)
        output_lang = Lang(lang2)

    return input_lang, output_lang, pairs

In [12]:
def filterPair(p):    
    return len(p[0].split(' ')) < MAX_LENGTH and \
        len(p[1].split(' ')) < MAX_LENGTH
#     if len(p) == 1:
#         print(p)


def filterPairs(pairs):
    return [pair for pair in pairs if filterPair(pair)]

In [13]:
def indexesFromSentence(lang, sentence):
    return [lang.word2index[word] for word in sentence.split(' ')]
#     return [lang.word2index[word[1:]] if word[0]=='-' and len(word)>1  else lang.word2index[word[1:]] for word  in re.findall(r"[\w'-]+|[.,!?;]", sentence)]
#     lang_list = []
#     for word in re.findall(r"[\w'-]+|[.,!?;]", sentence):
#         if word[0] == '-' and len(word) > 1:
#             lang_list.append(lang.word2index[word[1:]])
#         elif word[-1] == '-' and len(word) > 1:
#             lang_list.append(lang.word2index[word[:-1]])
#         else:
#             lang_list.append(lang.word2index[word])
#     return lang_list

# re.findall(r"[\w']+|[.,!?;]", sentence)


def tensorFromSentence(lang, sentence):
    indexes = indexesFromSentence(lang, sentence)
    indexes.append(EOS_token)
    return torch.tensor(indexes, dtype=torch.long).view(-1, 1)


def tensorsFromPair(pair):
    input_tensor = tensorFromSentence(input_lang, pair[0])
    target_tensor = tensorFromSentence(output_lang, pair[1])
    return (input_tensor, target_tensor)

In [14]:
# # fi_embeddings.wv.vocab['mies'].index
# # input_sentence[0].item()
# training_pairs = [tensorsFromPair(random.choice(pairs)) for i in range(10)]
# input_sentence, output_sentence = training_pairs[0]
# # print(output_lang.index2word[output_sentence[7145].item()])
# print(output_lang.word2index['пополека'])
# print(mk_embeddings.wv.vocab['пополека'].index)

In [15]:
def prepareData(lang1, lang2, reverse=False):
    input_lang, output_lang, pairs = readLangs(lang1, lang2, reverse)
    print("Read %s sentence pairs" % len(pairs))
    print("Counting words...")
    pairs = filterPairs(pairs)
    print("Trimmed to %s sentence pairs" % len(pairs))
    for pair in pairs:
        input_lang.addSentence(pair[0])
        output_lang.addSentence(pair[1])
    print("Counted words:")
    print(input_lang.name, input_lang.n_words)
    print(output_lang.name, output_lang.n_words)
    return input_lang, output_lang, pairs


# input_lang, output_lang, pairs = prepareData('fin_train', 'mk_train', False)
# input_lang, output_lang, pairs = prepareData('fin_train', 'mk_train', False)

input_lang, output_lang, pairs = prepareData('fin', 'mk', False)

Reading lines...
Read 2392808 sentence pairs
Counting words...
Trimmed to 1915404 sentence pairs
Counted words:
fin 531025
mk 340779


In [16]:
pairs_train, pairs_test = train_test_split(pairs, test_size=0.05, random_state=42)

In [17]:
# mk_word =  max(output_lang.word2index.items(), key=operator.itemgetter(1))[0]
# mk_max_index = output_lang.word2index[mk_word] + 1

# fi_word =  max(input_lang.word2index.items(), key=operator.itemgetter(1))[0]
# fi_max_index = input_lang.word2index[fi_word] + 1

In [18]:
from torch.nn.utils.rnn import pad_sequence

def collate(list_of_samples):
    # sort a list by sequence length
    list_of_samples.sort(key=lambda x: len(x[0]), reverse=True)

    input_seqs, output_seqs = zip(*list_of_samples)
    input_seq_lengths = [len(seq) for seq in input_seqs]
    output_seq_lengths = [len(seq) for seq in output_seqs]

    padding_value = 0
    
    pad_input_seqs = pad_sequence(input_seqs, batch_first=False, padding_value=padding_value)
    pad_output_seqs = []
    for i in output_seqs:
        padded = i.new_zeros(max(output_seq_lengths) - i.size(0))
        pad_output_seqs.append(torch.cat((i, padded.view(-1, 1)), dim=0))
    
    pad_output_seqs = torch.stack(pad_output_seqs)
    pad_output_seqs = pad_output_seqs.permute(1, 0, 2)

    return pad_input_seqs, input_seq_lengths, pad_output_seqs, output_seq_lengths

In [19]:
from torch.nn.utils.rnn import pack_padded_sequence, pad_packed_sequence

class Encoder(nn.Module):
    def __init__(self, dictionary_size, hidden_size, dropout_p=0.2, num_layers=1):
        super(Encoder, self).__init__()
        self.hidden_size = hidden_size
        self.num_layers = num_layers

        self.embedding = nn.Embedding(dictionary_size, hidden_size)
#         self.embedding = nn.Embedding.from_pretrained(fi_weights)
#         self.embedding.requires_grad = True
        self.lstm = nn.LSTM(hidden_size, 
                          hidden_size, 
                          num_layers=self.num_layers,
                          bidirectional=True
                         )

    def forward(self, pad_seqs, seq_lengths, hidden):
        embedded = self.embedding(pad_seqs).squeeze(dim=2)
        packed = pack_padded_sequence(embedded, seq_lengths)
        outputs, hidden = self.lstm(packed, hidden)
        outputs = pad_packed_sequence(outputs)[0]
        return outputs, hidden

    def init_hidden(self, batch_size=1, device=device):
        return (torch.zeros(2*self.num_layers, batch_size, self.hidden_size, device=device), torch.zeros(2*self.num_layers, batch_size, self.hidden_size, device=device))

In [20]:
class Decoder(nn.Module):
    def __init__(self, hidden_size, output_dictionary_size, dropout_p=0.2, num_layers=1):
        super(Decoder, self).__init__()
        self.hidden_size = hidden_size
        self.num_layers = num_layers
        self.embedding = nn.Embedding(output_dictionary_size, hidden_size)
#         self.embedding = nn.Embedding.from_pretrained(mk_weights)
#         self.embedding.requires_grad = True
        self.lstm = nn.LSTM(hidden_size, 
                          hidden_size, 
                          num_layers=self.num_layers,
                          bidirectional=False
                         )
        self.lin = nn.Linear(hidden_size, output_dictionary_size)
        self.softmax = nn.LogSoftmax(dim=2)

    def forward(self, hidden, pad_target_seqs=None, teacher_forcing=False):
        batch_size = hidden[1].size(1)
        prev_word = torch.tensor(SOS_token * np.ones((1, batch_size)), device=device, dtype=torch.int64)
        max_length = pad_target_seqs.size(0) if pad_target_seqs is not None else MAX_LENGTH
        outputs = []
        for t in range(max_length):
            prev_word = prev_word.view(1, -1)
            output = self.embedding(prev_word)
            output, hidden = self.lstm(output, hidden)
            output = self.softmax(self.lin(output))
    
            outputs.append(output)

            if teacher_forcing:
                prev_word = pad_target_seqs[t]
            else:
                topv, topi = output[0, :].topk(1)
                prev_word = topi.squeeze().detach()

        outputs = torch.cat(outputs, dim=0)
        return outputs, hidden

#     def init_hidden(self, batch_size, device=device):
#         return torch.zeros(1*self.num_layers, batch_size, self.hidden_size, device=device)

In [21]:
hidden_size = 256
# hidden_size = 300
encoder = Encoder(input_lang.n_words, hidden_size, dropout_p=0.2).to(device)
decoder = Decoder(hidden_size, output_lang.n_words, dropout_p=0.2).to(device)
# encoder = Encoder(fi_max_index, hidden_size, dropout_p=0.3).to(device)
# decoder = Decoder(hidden_size, mk_max_index, dropout_p=0.3).to(device)

In [22]:
criterion = nn.NLLLoss(reduction='none')

def compute_loss(decoder_outputs, pad_target_seqs, padding_value=0):  
    n = 0
    loss = 0
    for i in range(decoder_outputs.shape[1]):
        p_arr = []
        t_arr = []
        p = decoder_outputs[:, i, :]
        t = pad_target_seqs[:, i].squeeze()
        for word in range(len(t)):
            if t[word] != padding_value:
                p_arr.append(p[word])
                t_arr.append(t[word])
                n += 1
        p_arr = torch.stack(p_arr)
        t_arr = torch.stack(t_arr)
        loss += criterion(p_arr, t_arr).sum()
    loss = loss / n

    return loss

In [23]:
n_epochs = 10
teacher_forcing_ratio = 0.5

encoder_optimizer = optim.Adam(encoder.parameters(), lr=0.01)
decoder_optimizer = optim.Adam(decoder.parameters(), lr=0.01)

In [24]:
training_pairs = [tensorsFromPair(random.choice(pairs_train)) for i in range(10)]        
pairs_batch_train = DataLoader(dataset=training_pairs,
                 batch_size=128,
                 shuffle=True,
                 collate_fn=collate,
                 pin_memory=True)

In [27]:
if skip_training == False:
    for epoch in range(n_epochs):
        testing_pairs = [tensorsFromPair(random.choice(pairs_test)) for i in range(1000)]
        pairs_batch_test = DataLoader(dataset=testing_pairs,
                 batch_size=128,
                 shuffle=True,
                 collate_fn=collate,
                 pin_memory=True)

        complete_loss = 0
        for i, batch in enumerate(pairs_batch_train):
            pad_input_seqs, input_seq_lengths, pad_target_seqs, target_seq_lengths = batch
            batch_size = pad_input_seqs.size(1)
            pad_input_seqs, pad_target_seqs = pad_input_seqs.to(device), pad_target_seqs.to(device)

            encoder_hidden = encoder.init_hidden(batch_size, device)
            encoder_optimizer.zero_grad()
            decoder_optimizer.zero_grad()

            # Encode input sequence
            _, encoder_hidden = encoder(pad_input_seqs, input_seq_lengths, encoder_hidden)
            # Decode using target sequence for teacher forcing
            decoder_hidden = (encoder_hidden[0].mean(0, keepdim=True),encoder_hidden[1].mean(0, keepdim=True))
#             decoder_hidden = decoder_hidden[None, :, :]
#             decoder_hidden = torch.mean(decoder_hidden[0], 0)[0]
#             teacher_forcing = True if random.random() < teacher_forcing_ratio else False
            if random.random() < teacher_forcing_ratio:
                teacher_forcing = True
            else:
                teacher_forcing = False
                
            decoder_outputs, decoder_hidden = decoder(decoder_hidden, pad_target_seqs, teacher_forcing=teacher_forcing)
            loss = compute_loss(decoder_outputs, pad_target_seqs, padding_value=0)
            loss.backward()
            
            complete_loss += loss

            encoder_optimizer.step()
            decoder_optimizer.step()
            
        print('[Epoch: %d] loss: %.4f' % (epoch + 1, complete_loss / len(pairs_batch_train)))
        with open("loss.txt", "a") as f:
            f.write(str(complete_loss.item() / len(pairs_batch_train)) + '\n')  
            
        test_loss = 0
        for i, batch in enumerate(pairs_batch_test):
            pad_input_seqs, input_seq_lengths, pad_target_seqs, target_seq_lengths = batch
            batch_size = pad_input_seqs.size(1)
            pad_input_seqs, pad_target_seqs = pad_input_seqs.to(device), pad_target_seqs.to(device)

            encoder_hidden = encoder.init_hidden(batch_size, device)

            # Encode input sequence
            _, encoder_hidden = encoder(pad_input_seqs, input_seq_lengths, encoder_hidden)
            # Decode using target sequence for teacher forcing
            decoder_hidden = (encoder_hidden[0].mean(0, keepdim=True),encoder_hidden[1].mean(0, keepdim=True))

            decoder_outputs, decoder_hidden = decoder(decoder_hidden, pad_target_seqs, teacher_forcing=False)
            loss = compute_loss(decoder_outputs, pad_target_seqs, padding_value=0)
            test_loss += loss
            
        print('[Epoch: %d] test loss: %.4f' % (epoch + 1, test_loss / len(pairs_batch_test)))
        with open("test_loss.txt", "a") as f:
            f.write(str(test_loss.item() / len(pairs_batch_test)) + '\n')
        
            
        torch.save(encoder, 'model/encoder')
        torch.save(decoder, 'model/decoder')
    
    print('Final loss is: %f' % (complete_loss.item() / len(pairs_batch_train)))
    print('Finished Training')

[Epoch: 1] loss: 6.6575


Exception ignored in: <bound method _DataLoaderIter.__del__ of <torch.utils.data.dataloader._DataLoaderIter object at 0x7f6860cf4be0>>
Traceback (most recent call last):
  File "/u/15/porjazd1/unix/.local/lib/python3.6/site-packages/torch/utils/data/dataloader.py", line 715, in __del__
    def __del__(self):
KeyboardInterrupt


RuntimeError: $ Torch: not enough memory: you tried to allocate 1GB. Buy new RAM! at /pytorch/aten/src/TH/THGeneral.cpp:201

In [286]:
if skip_training == False:
    torch.save(encoder, 'model/encoder')
    torch.save(decoder, 'model/decoder')
else:
    encoder = torch.load('model/encoder')
    encoder.eval()
    
    decoder = torch.load('model/decoder')
    decoder.eval()

/u/15/porjazd1/unix/.local/lib/python3.6/site-packages/torch/serialization.py:250: UserWarning: Couldn't retrieve source code for container of type Encoder. It won't be checked for correctness upon loading.
  "type " + obj.__name__ + ". It won't be checked "
/u/15/porjazd1/unix/.local/lib/python3.6/site-packages/torch/serialization.py:250: UserWarning: Couldn't retrieve source code for container of type Decoder. It won't be checked for correctness upon loading.
  "type " + obj.__name__ + ". It won't be checked "


In [114]:
def evaluate(input_seq):
    with torch.no_grad():
        input_length = input_seq.size(0)
        batch_size = 1

        encoder_hidden = encoder.init_hidden(batch_size, device)
        input_seq = input_seq.view(-1, 1, 1).to(device)
        encoder_output, encoder_hidden = encoder(input_seq, [input_length], encoder_hidden)

        decoder_hidden = (encoder_hidden[0].mean(0, keepdim=True),encoder_hidden[1].mean(0, keepdim=True))
        decoder_outputs, decoder_hidden = decoder(decoder_hidden, pad_target_seqs=None, teacher_forcing=False)

        output_seq = []
        for t in range(decoder_outputs.size(0)):
            topv, topi = decoder_outputs[t].data.topk(1)
            output_seq.append(topi.item())
            if topi.item() == EOS_token:
                break

    return output_seq

In [116]:
# Evaluate random sentences from the training set
print('\nEvaluate on training data:')
print('-----------------------------')
for i in range(5):
    input_sentence, target_sentence = training_pairs[np.random.choice(len(training_pairs))]
    print('>', ' '.join(input_lang.index2word[i.item()] for i in input_sentence))
    print('=', ' '.join(output_lang.index2word[i.item()] for i in target_sentence))
    output_sentence = evaluate(input_sentence)
    print('<', ' '.join(output_lang.index2word[i] for i in output_sentence))
    print('')


Evaluate on training data:
-----------------------------
> entä aukio? EOS
= а сквер? EOS
< не EOS

> saat vielä osasi . tulkaahan, tytöt. EOS
= ќе си го добиете.ајде девојки. EOS
< да, да EOS

> kaikki on ollut sekaisin cassien tultua tänne. EOS
= откако дојде кеси настана хаос. EOS
< да, да EOS

> entä aukio? EOS
= а сквер? EOS
< не EOS

> laita se pois. EOS
= врати го. EOS
< не EOS



In [121]:
# reference = [['the', 'quick', 'brown', 'fox', 'jumped', 'over', 'the', 'lazy', 'dog']]
# candidate = ['the', 'quick', 'brown', 'fox', 'jumped', 'over', 'the', 'lazy', 'dog']
# score = sentence_bleu(reference, candidate)
# print(score)

In [118]:
def calculate_bleu(input_seq, target_seq):
    reference = []
    candidate = []
    
    input_sentence = evaluate(input_seq)
    target_sentence = evaluate(target_seq)
    
    for word in input_sentence:
        reference.append(word)
    
    for word in target_sentence:
        candidate.append(word)
    
    reference = [reference]
    
    return sentence_bleu(reference, candidate)